# 概述

[House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview)

目標: 根據房屋特徵來預測房屋的最終價格。

# 載入套件

In [1]:
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from flaml import AutoML

import warnings
warnings.filterwarnings('ignore')

# 載入資料集

In [2]:
# 讀取訓練資料集和測試資料集
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

print('訓練資料集和測試資料集成功!\n')
print(f'訓練資料集: {train_data.shape[0]} rows, {train_data.shape[1]} columns')
print(f'測試資料集: {test_data.shape[0]} rows, {test_data.shape[1]} columns')

訓練資料集和測試資料集成功!

訓練資料集: 1460 rows, 81 columns
測試資料集: 1459 rows, 80 columns


# 資料處理

## 處理缺失值

In [3]:
def impute_dataframe(dataframe):
    # 複製資料，以免直接修改原始資料
    df = dataframe.copy()

    # 缺失值的類別型欄位
    no_have_cat = [
        "Alley",  # 巷道類型
        "BsmtQual",  # 地下室質量
        "BsmtCond",  # 地下室條件
        "BsmtExposure",  # 地下室暴露程度
        "BsmtFinType1",  # 地下室完成類型1
        "BsmtFinType2",  # 地下室完成類型2
        "FireplaceQu",  # 壁爐質量
        "GarageType",  # 車庫類型
        "GarageFinish",  # 車庫完成度
        "GarageQual",  # 車庫質量
        "GarageCond",  # 車庫條件
        "PoolQC",  # 游泳池質量
        "Fence",  # 圍欄質量
        "MiscFeature",  # 其他特徵
        "MasVnrType",  # 砌體飾面類型
    ]

    # 缺失值的數值型欄位
    no_have_num = [
        "MasVnrArea",  # 砌體飾面面積
        "GarageYrBlt",  # 車庫建造年份
        "GarageArea",  # 車庫面積
        "GarageCars",  # 車庫可容納的汽車數量
        "BsmtFinSF1",  # 地下室完成面積1
        "BsmtFinSF2",  # 地下室完成面積2
        "BsmtUnfSF",  # 未完成的地下室面積
        "TotalBsmtSF",  # 地下室總面積
        "BsmtFullBath",  # 地下室完整浴室數量
        "BsmtHalfBath",  # 地下室半浴室數量
    ]

    # 填補類別型欄位的缺失值為"Absent"
    df[no_have_cat] = df[no_have_cat].fillna("Absent")

    # 填補數值型欄位的缺失值為0
    df[no_have_num] = df[no_have_num].fillna(0)

    # 根據 Neighborhood 分組後，使用中位數填補 LotFrontage 的缺失值
    df["LotFrontage"] = df["LotFrontage"].fillna(
        df.groupby("Neighborhood")["LotFrontage"].transform("median")
    )

    # 根據 MSSubClass 分組後，使用眾數填補 MSZoning 的缺失值
    df["MSZoning"] = df.groupby("MSSubClass")["MSZoning"].transform(
        lambda x: x.fillna(x.mode()[0])
    )

    # 使用眾數填補其他類別型欄位的缺失值
    for col in [
        "Electrical",  # 電力系統
        "Utilities",  # 公用設施
        "Exterior1st",  # 外牆材料1
        "Exterior2nd",  # 外牆材料2
        "SaleType",  # 銷售類型
        "KitchenQual",  # 廚房質量
    ]:
        df[col] = df.groupby(["Neighborhood", "OverallQual"])[col].transform(
            lambda x: x.fillna(x.mode()[0])
        )

    # 填補 Functional 缺失值為 "Typ"
    df["Functional"] = df["Functional"].fillna("Typ")

    # 填補 MSSubClass 並轉為類別型態，缺失值填補為 "Absent"
    df["MSSubClass"] = df["MSSubClass"].astype("object").fillna("Absent")

    return df

# 特徵工程

In [4]:
def introduce_features(dataframe):
    # 複製資料，以免直接修改原始資料
    df = dataframe.copy()

    # 一樓面積與生活面積的乘積
    df["NEW_1st*GrLiv"] = df["1stFlrSF"] * df["GrLivArea"]

    # 車庫面積與生活面積的乘積
    df["NEW_Garage*GrLiv"] = (df["GarageArea"] * df["GrLivArea"])

    # 總樓層面積 (一樓與二樓面積相加)
    df["NEW_TotalFlrSF"] = df["1stFlrSF"] + df["2ndFlrSF"]

    # 地下室完成的總面積
    df["NEW_TotalBsmtFin"] = df["BsmtFinSF1"] + df["BsmtFinSF2"]

    # 門廊的總面積 (包括開放式門廊、封閉式門廊、屏風門廊、三季門廊、木甲板)
    df["NEW_PorchArea"] = df["OpenPorchSF"] + df["EnclosedPorch"] + df["ScreenPorch"] + df["3SsnPorch"] + df["WoodDeckSF"]

    # 房屋的總面積 (樓層面積與地下室面積相加)
    df["NEW_TotalHouseArea"] = df["NEW_TotalFlrSF"] + df["TotalBsmtSF"]
    df["NEW_TotalSqFeet"] = df["GrLivArea"] + df["TotalBsmtSF"]

    # 生活面積與地塊面積的比率
    df["NEW_LotRatio"] = df["GrLivArea"] / df["LotArea"]

    # 房屋總面積與地塊面積的比率
    df["NEW_RatioArea"] = df["NEW_TotalHouseArea"] / df["LotArea"]

    # 車庫面積與地塊面積的比率
    df["NEW_GarageLotRatio"] = df["GarageArea"] / df["LotArea"]

    # 砌體飾面面積與房屋總面積的比率
    df["NEW_MasVnrRatio"] = df["MasVnrArea"] / df["NEW_TotalHouseArea"]

    # 地塊剩餘面積 (地塊面積減去一樓、車庫和門廊的面積)
    df["NEW_DifArea"] = df["LotArea"] - df["1stFlrSF"] - df["GarageArea"] - df["NEW_PorchArea"] - df["WoodDeckSF"]

    # 總質量 (OverallQual 與 OverallCond 的乘積)
    df["NEW_OverallGrade"] = df["OverallQual"] * df["OverallCond"]

    # 房屋翻修年份與建造年份的差值
    df["NEW_Restoration"] = df["YearRemodAdd"] - df["YearBuilt"]

    # 房屋的年齡 (出售年份與建造年份的差值)
    df["NEW_HouseAge"] = df["YrSold"] - df["YearBuilt"]

    # 翻修年齡 (出售年份與翻修年份的差值)
    df["NEW_RestorationAge"] = df["YrSold"] - df["YearRemodAdd"]

    # 車庫年齡 (車庫建造年份與房屋建造年份的差值)
    df["NEW_GarageAge"] = df["GarageYrBlt"] - df["YearBuilt"]

    # 車庫翻修年齡 (車庫建造年份與翻修年份的絕對差值)
    df["NEW_GarageRestorationAge"] = np.abs(df["GarageYrBlt"] - df["YearRemodAdd"])

    # 車庫出售年齡 (出售年份與車庫建造年份的差值)
    df["NEW_GarageSold"] = df["YrSold"] - df["GarageYrBlt"]

    # 地下室完成面積佔總地下室面積的比率
    df["NEW_BsmtFinSFRatio"] = df["BsmtFinSF1"] / df["TotalBsmtSF"]
    df["NEW_BsmtFinSFRatio"] = df["NEW_BsmtFinSFRatio"].fillna(df["NEW_BsmtFinSFRatio"].median())

    # 平均房間大小 (生活面積除以房間總數)
    df["NEW_AvgRoomSize"] = df["GrLivArea"] / df["TotRmsAbvGrd"]

    # 總浴室數量 (包括完整與半浴室，半浴室計為 0.5)
    df["NEW_TotalBath"] = df["FullBath"] + df["BsmtFullBath"] + 0.5 * (df["HalfBath"] + df["BsmtHalfBath"])

    # 總房間數 (總房間數加上地下室的房間數)
    df["NEW_TotalRooms"] = df["TotRmsAbvGrd"] + df["BedroomAbvGr"]

    # 房屋面積與地塊面積的比率
    df["NEW_BuildingToLotRatio"] = df["NEW_TotalHouseArea"] / df["LotArea"]

    # 戶外空間比率 (門廊面積佔房屋總面積的比率)
    df["NEW_OutdoorSpaceRatio"] = df["NEW_PorchArea"] / df["NEW_TotalHouseArea"]

    # 完成面積比率 (樓層與地下室的完成面積佔房屋總面積的比率)
    df["NEW_FinishingRatio"] = (df["NEW_TotalFlrSF"] + df["NEW_TotalBsmtFin"]) / df["NEW_TotalHouseArea"]

    # 轉換銷售月份和年份為日期格式
    df['date'] = pd.to_datetime(df['MoSold'].astype('str') + '.' + df['YrSold'].astype('str'), format='%m.%Y').dt.strftime('%m.%Y')

    return df

# Encoding

In [5]:
# 定義序數特徵對應的等級順序
ordinal = {
    "PoolQC": ["Fa", "TA", "Gd", "Ex"],  # 泳池質量（從低到高）
    "Fence": ["MnWw", "GdWo", "MnPrv", "GdPrv"],  # 圍欄質量
    "FireplaceQu": ["Po", "Fa", "TA", "Gd", "Ex"],  # 壁爐質量
    "GarageFinish": ["Unf", "RFn", "Fin"],  # 車庫完成程度
    "GarageQual": ["Po", "Fa", "TA", "Gd", "Ex"],  # 車庫質量
    "GarageCond": ["Po", "Fa", "TA", "Gd", "Ex"],  # 車庫狀況
    "BsmtQual": ["Po", "Fa", "TA", "Gd", "Ex"],  # 地下室質量
    "BsmtCond": ["Po", "Fa", "TA", "Gd", "Ex"],  # 地下室狀況
    "BsmtExposure": ["No", "Mn", "Av", "Gd"],  # 地下室光照程度
    "BsmtFinType1": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],  # 地下室完成類型1
    "BsmtFinType2": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],  # 地下室完成類型2
    "KitchenQual": ["Po", "Fa", "TA", "Gd", "Ex"],  # 廚房質量
    "LotShape": ["IR3", "IR2", "IR1", "Reg"],  # 地塊形狀
    "LandSlope": ["Sev", "Mod", "Gtl"],  # 土地坡度
    "ExterQual": ["Po", "Fa", "TA", "Gd", "Ex"],  # 外部質量
    "ExterCond": ["Po", "Fa", "TA", "Gd", "Ex"],  # 外部狀況
    "HeatingQC": ["Po", "Fa", "TA", "Gd", "Ex"],  # 供暖質量
    "PavedDrive": ["N", "P", "Y"],  # 鋪設的車道（無、部分、全部）
    "Functional": ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"]  # 功能狀態（從不良到典型）
}

# 定義名目特徵
nominal = [
    "MSZoning", "Electrical", "Exterior1st", "Exterior2nd", "SaleType", "Street",
    "LandContour", "LotConfig", "Neighborhood", "Condition1", "Condition2",
    "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Foundation", "Heating",
    "CentralAir", "SaleCondition", "MiscFeature", "Alley", "GarageType", "MasVnrType",
    "MSSubClass", "Utilities"
]

# 標籤編碼名目類別的函數
def label_encode_nominal_columns(dataframe, nominal_columns):
    """
    將指定的名目類別欄位進行標籤編碼。

    Parameters:
    dataframe (pd.DataFrame): 輸入的資料。
    nominal_columns (list): 需要標籤編碼的名目類別欄位列表。

    Returns:
    pd.DataFrame: 進行標籤編碼後的數據。
    """
    le = LabelEncoder()  # 創建標籤編碼器
    df = dataframe.copy()  # 複製資料避免直接修改原數據

    # 將名目欄位轉換為字串並進行標籤編碼
    for col in nominal_columns:
        df[col] = le.fit_transform(df[col].astype(str))

    return df

# 標籤編碼序數類別的函數
def label_encode_ordinal_columns(dataframe, ordinal_columns):
    """
    根據提供的順序對指定的序數類別欄位進行標籤編碼，對未知值插入 -1。

    Parameters:
    dataframe (pd.DataFrame): 輸入的資料。
    ordinal_columns (dict): 序數類別欄位字典，鍵為欄位名稱，值為對應的順序列表。

    Returns:
    pd.DataFrame: 進行標籤編碼後的資料。
    """
    df = dataframe.copy()

    for col, order in ordinal_columns.items():
        encoder = OrdinalEncoder(categories=[order], handle_unknown='use_encoded_value', unknown_value=-1)
        df[col] = encoder.fit_transform(df[[col]]).flatten()

    return df

# One-Hot編碼的函數
def one_hot_encoder(dataframe, categorical_cols):
    """
    將指定的類別欄位進行 One-Hot 編碼。

    Parameters:
    dataframe (pd.DataFrame): 輸入的資料。
    categorical_cols (list): 需要進行 One-Hot 編碼的類別欄位列表。

    Returns:
    pd.DataFrame: 進行 One-Hot 編碼後的資料。
    """
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=True, dtype=int)
    return dataframe

# 資料預處理

In [6]:
def preprocess_data(df_train, df_test):
    # 合併訓練集和測試集進行統一的預處理
    df = pd.concat([df_train, df_test], axis=0)
    
    df = impute_dataframe(df)
    df = introduce_features(df)
    df = label_encode_ordinal_columns(df, ordinal)
    df = label_encode_nominal_columns(df, nominal)
    df = one_hot_encoder(df, nominal)

    df_train = df.query("SalePrice.notnull()")
    y_train = np.log1p(df_train["SalePrice"])
    x_train = df_train.drop("SalePrice", axis=1)
    x_test = df.query("SalePrice.isna()").drop("SalePrice", axis=1)

    # 將訓練數據集進行 80% 訓練集和 20% 驗證集的切分
    X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

    # 保存欄位名稱，以便之後轉換後能夠保留原本的欄位
    column_names = X_train.columns

    # 使用 RobustScaler 對數據進行標準化處理
    rb = RobustScaler()
    X_train = rb.fit_transform(X_train)
    X_val = rb.transform(X_val)
    x_test = rb.transform(x_test)

    # 將縮放後的數據轉回 DataFrame 格式，保留欄位名稱
    X_train = pd.DataFrame(X_train, columns=column_names)
    X_val = pd.DataFrame(X_val, columns=column_names)
    X_test = pd.DataFrame(x_test, columns=column_names)

    return X_train, X_val, y_train, y_val, X_test

# 從測試集中保存 Id 欄位
test_ids = test_data['Id'].copy()

# 對訓練集和測試集進行預處理，移除 Id 欄位
X_train, X_val, y_train, y_val, X_test = preprocess_data(train_data.drop(columns=['Id']), test_data.drop(columns=['Id']))

# 訓練模型

In [7]:
# 使用FLAML進行自動化機器學習
automl = AutoML()
automl_settings = {
    "time_budget": 21600,
    "metric": 'rmse',
    "task": 'regression',
    "log_file_name": "flaml.log",
}

# 訓練模型
automl.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **automl_settings)

[flaml.automl.logger: 09-09 16:42:53] {1680} INFO - task = regression
[flaml.automl.logger: 09-09 16:42:53] {1688} INFO - Data split method: uniform
[flaml.automl.logger: 09-09 16:42:53] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 09-09 16:42:53] {1789} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 09-09 16:42:53] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 09-09 16:42:53] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 09-09 16:42:53] {2345} INFO - Estimated sufficient time budget=1323s. Estimated necessary time budget=11s.
[flaml.automl.logger: 09-09 16:42:53] {2392} INFO -  at 0.4s,	estimator lgbm's best error=0.3430,	best estimator lgbm's best error=0.3430
[flaml.automl.logger: 09-09 16:42:53] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 09-09 16:42:53] {2392} INFO -  at 0.4s,	estimator lgbm's best error=0.34

# 評估模型

均方根誤差(Root Mean Squared Error, RMSE)是一個常用的評估回歸模型準確度的指標。

$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

In [8]:
# 評估模型
y_pred = automl.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f"Validation RMSE: {rmse}")

Validation RMSE: 0.12497840600187622


# 產生提交檔

![kaggle-submission-SVR-20240818](./images/kaggle-house-prices-flaml-2024-09-09.png)

In [9]:
# 預測測試集
test_predictions = automl.predict(X_test)

# 將預測結果轉換回原始規模
test_predictions = np.expm1(test_predictions)

# 產生Kaggle提交檔案
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions})
date = datetime.datetime.today().strftime('%Y-%m-%d')
submission_path = f'./data/Submission-flaml-{date}.csv'
submission.to_csv(submission_path, index=False)
print(f"Submission saved successfully as: {submission_path}")

Submission saved successfully as: ./data/Submission-flaml-2024-09-09.csv
